Importing the libraries

In [2]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

<ipython-input-2-ed7c4f6abf45>:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
data = pd.read_csv("movies.csv")

In [4]:
# show the first five rows
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [5]:
# show the num of rows and columns
data.shape

(4803, 24)

In [6]:
# show more information
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [7]:
# selecting the relevant features
selected = ['genres','keywords','tagline','cast','director']

In [8]:
# replacing null values
for feature in selected:
    data[feature] = data[feature].fillna('')

In [9]:
# combined the 5 features
combined = data['genres'] + " " + data['keywords'] + " " + data['tagline'] + " " + data['cast'] + " " + data['director']
combined

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [10]:
vectorizer = TfidfVectorizer()

In [11]:
vectorized = vectorizer.fit_transform(combined)

In [12]:
similarity = cosine_similarity(vectorized)

Build a recommender system

In [48]:
def recommend_movies(movie_name, data, similarity):
    # Convert movie name to string and handle potential input issues
    movie_name = str(movie_name).strip()
    
    # List all movies in the dataset
    listed_movies = data['title'].tolist()
    
    # Find the closest match for the input movie name
    try:
        closed_matches = difflib.get_close_matches(movie_name, listed_movies, n=1, cutoff=0.6)
        
        # Check if any close matches were found
        if not closed_matches:
            print(f"No close match found for '{movie_name}'. Please check the movie name.")
            return []
        
        # Get the first close match
        close_match = closed_matches[0]
        
        # Get the index of the matched movie
        movie_index = data[data['title'] == close_match]['index'].values[0]
        
        # Calculate similarity scores
        similarity_score = list(enumerate(similarity[movie_index]))
        
        # Sort movies based on similarity score
        sorted_movie = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        
        # Print recommended movies
        print(f'Movies suggested based on {close_match}:\n')
        for i, (index, score) in enumerate(sorted_movie[1:31], 1):  # Skip the first one (the movie itself)
            name_of_movie = data[data['index'] == index]['title'].values[0]
            print(f'{i}. {name_of_movie}')
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [49]:
recommend_movies("batman",data,similarity)

Movies suggested based on Batman:

1. Batman Returns
2. Batman & Robin
3. The Dark Knight Rises
4. Batman Begins
5. The Dark Knight
6. A History of Violence
7. Superman
8. Beetlejuice
9. Bedazzled
10. Mars Attacks!
11. The Sentinel
12. Planet of the Apes
13. Man of Steel
14. Suicide Squad
15. The Mask
16. Salton Sea
17. Spider-Man 3
18. The Postman Always Rings Twice
19. Hang 'em High
20. Spider-Man 2
21. Dungeons & Dragons: Wrath of the Dragon God
22. Superman Returns
23. Jonah Hex
24. Exorcist II: The Heretic
25. Superman II
26. Green Lantern
27. Superman III
28. Something's Gotta Give
29. Reds
30. Batman Forever
